In [1]:
from func import *

In [2]:
train_df = load_csvs_from_ftp_to_df(provided_local_dir="/")

In [3]:
positive_col_list = ["BedroomsTotal",
                     "BathroomsTotalInteger",
                     "LotSizeAcres",
                     "LotSizeArea",
                     "LotSizeSquareFeet"]
non_negative_col_list = ["ParkingTotal"]
default_drop_col = ["ListingId",
                    "ListingKey",
                    "ListingKeyNumeric",
                    "ListPrice",
                    "OriginalListPrice"]

In [4]:
(train_df_clean,
 knn_model,
 train_df_ref,
 reference_col_list,
 k_means_model,
 col_drop_list,
 scaler,
 cols_with_na,
 save_name) = pre_process(train_df,
                             price_col="ClosePrice",
                             default_threshold=0.8,
                             col_drop=default_drop_col,
                             positive_col_list=positive_col_list,
                             non_negative_col_list=non_negative_col_list,
                             flag_col_list=["Flooring"],
                             yn_col_list=["AttachedGarageYN",
                                          "ViewYN",
                                          "NewConstructionYN",
                                          "PoolPrivateYN",
                                          "FireplaceYN"],
                             knn_k=3,
                             knn_model=None,
                             train_df_ref=None,
                             reference_col_list=None,
                             num_clusters=10,
                             clustering_method="k-means",
                             clustering_model=None,
                             scaler_method="robust",
                             scaler=None,
                             save_name="processed",
                             train_data=True,
                             save=True,
                             cols_with_na=None
                             )

C:\My_Programs\Python_Project\IDX_Intern_self\func.py:261: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[val + "YN"] = df[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern_self\func.py:261: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[val + "YN"] = df[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern_self\func.py:261: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beh

In [5]:
test_df = load_csvs_from_ftp_to_df(provided_local_dir="/",
                                   date_range=range(12, 13))
col_drop_list.extend(default_drop_col)

In [7]:
test_df_clean, _, _, _, _, _, _, _, _ = pre_process(test_df,
                                                 price_col="ClosePrice",
                                                 default_threshold=0.8,
                                                 col_drop=col_drop_list,
                                                 positive_col_list=positive_col_list,
                                                 non_negative_col_list=non_negative_col_list,
                                                 flag_col_list=["Flooring"],
                                                 yn_col_list=["AttachedGarageYN",
                                                              "ViewYN",
                                                              "NewConstructionYN",
                                                               "PoolPrivateYN",
                                                               "FireplaceYN"],
                                                 knn_k=3,
                                                 knn_model=knn_model,
                                                 train_df_ref=train_df_ref,
                                                 reference_col_list=reference_col_list,
                                                 num_clusters=10,
                                                 clustering_method="k-means",
                                                 clustering_model=k_means_model,
                                                 scaler_method="robust",
                                                 scaler=scaler,
                                                 save_name=save_name,
                                                 train_data=False,
                                                 save=True,
                                                 cols_with_na=cols_with_na
                                                 )

C:\My_Programs\Python_Project\IDX_Intern_self\func.py:261: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[val + "YN"] = df[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern_self\func.py:261: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[val + "YN"] = df[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern_self\func.py:261: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beh

# Read Processed data

In [2]:
train_df = pd.read_csv("processed/train_data.csv")
test_df = pd.read_csv("processed/test_Data.csv")

# Train model (Ridge regression)

In [3]:
train_df["logClosePrice"] = np.log1p(train_df["ClosePrice"])
train_df.drop(columns=["ClosePrice"], inplace=True)
test_df["logClosePrice"] = np.log1p(test_df["ClosePrice"])
test_df.drop(columns=["ClosePrice"], inplace=True)

In [7]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=10.0, random_state=42)

ridge_result = fit_predict(
    train_df=train_df,
    test_df=test_df,
    model=model,
    col_drop_list=["ClosePrice"],
    target_col="logClosePrice",
    card_threshold=20,
    num_scaler="robust",
    smoothing=10,
    min_samples_leaf=20,
    log_transform=True
)
print(f"R²: {ridge_result['r2']:.4f}")
print(f"MdAPE (%): {ridge_result['mdape']:.2f}")

R²: 0.1365
MdAPE (%): 33.18


# Elastic Net (Ridge + Lasso hybrid)

In [8]:
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha=0.01, l1_ratio=0.5)

elastic_net_result = fit_predict(
    train_df=train_df,
    test_df=test_df,
    model=model,
    col_drop_list=["ClosePrice"],
    target_col="logClosePrice",
    card_threshold=20,
    num_scaler="robust",
    smoothing=10,
    min_samples_leaf=20,
    log_transform=True
)

print(f"R²: {elastic_net_result['r2']:.4f}")
print(f"MdAPE (%): {elastic_net_result['mdape']:.2f}")

R²: 0.7481
MdAPE (%): 12.39


# Histogram-based Gradient Boosting Regression Tree

In [13]:
from sklearn.ensemble import HistGradientBoostingRegressor

model = HistGradientBoostingRegressor(
    max_depth=6,
    learning_rate=0.05,
    max_iter=300,
    random_state=42,
)


hist_tree_result = fit_predict(
    train_df=train_df,
    test_df=test_df,
    model=model,
    col_drop_list=["ClosePrice"],
    target_col="logClosePrice",
    card_threshold=20,
    num_scaler="robust",
    smoothing=10,
    min_samples_leaf=20,
    log_transform=True
)

print(f"R²: {hist_tree_result['r2']:.4f}")
print(f"MdAPE (%): {hist_tree_result['mdape']:.2f}")

R²: -0.0189
MdAPE (%): 39.26


# Random forest

In [10]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    random_state=42,
    n_jobs=-1
)

rf_result = fit_predict(
    train_df=train_df,
    test_df=test_df,
    model=model,
    col_drop_list=["ClosePrice"],
    target_col="logClosePrice",
    card_threshold=20,
    num_scaler="robust",
    smoothing=10,
    min_samples_leaf=20,
    log_transform=True
)

print(f"R²: {rf_result['r2']:.4f}")
print(f"MdAPE (%): {rf_result['mdape']:.2f}")

R²: -0.0292
MdAPE (%): 39.17


# Multi-layer Perceptron

In [12]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(
    hidden_layer_sizes=(128, 64),
    activation="relu",
    alpha=1e-3,
    learning_rate_init=1e-3,
    max_iter=300,
    early_stopping=True,
    random_state=42
)

mlp_result = fit_predict(
    train_df=train_df,
    test_df=test_df,
    model=model,
    col_drop_list=["ClosePrice"],
    target_col="logClosePrice",
    card_threshold=20,
    num_scaler="robust",
    smoothing=10,
    min_samples_leaf=20,
    log_transform=True
)

print(f"R²: {mlp_result['r2']:.4f}")
print(f"MdAPE (%): {mlp_result['mdape']:.2f}")

R²: -5.2552
MdAPE (%): 19.56
